### Tr for Single Angle Bolted Through One Leg

Calculate factored tension resistance, $T_r$, of a single angle bolted through one leg.

![Angle bolted through one leg](images/bolted-single-angle.svg)

_Note:_ The figure shows 2 rows of 4 bolts through one leg.  The logic below works for
1 or 2 rows and almost any number of bolts per row.

In [1]:
from Designer import Designer, Param, Data, table_search, SST

In [2]:
Table6 = {'M16': (28,22),    # CSA S16-14 Table 6  - Minimum edge distances
          '3/4': (32,25),
          'M20': (34,26),
          '7/8': (38,28),
          'M22': (38,28),
          'M24': (42,30),
          '1':   (44,32),
         }

def get_Table6(size,col=1):
    assert col in [1,2], "Must select column 1 or 2 from Table 6."
    try:
        return Table6[size][col-1]
    except KeyError:
        pass
    raise KeyError('Cannot determine min edge distance for bolt size: "{}"'.format(size))

In [3]:
#           Leg:   g, g1
UG_Angle = {203: (115,75),    # Usual Gauges for Angles, CISC Handbook 11th ed., p. 6-173
           178: (100,65),
           152: (90,60),
           127: (75,50),
           102: (65,),
           89: (50,),
           76: (45,),
           64: (35,),
           51: (29,),
           44: (25,),
            }

def get_UG_Angle(leg):
    try:
        gs = UG_Angle[int(leg)]
        if len(gs) == 1:
            return gs[0],None
        return gs
    except KeyError:
        pass
    raise KeyError('Cannot determine gauges for angle leg of size "{}"'.format(leg))

In [4]:
BOLT_DIA = {'M16': 16.,
           '3/4': 25.4*3/4,
           'M20': 20.,
           '7/8': 25.4*7/8,
           'M22': 22.,
           'M24': 24.,
           '1': 25.4,
           }

HOLE_DIA = {'M16': 18.,
           '3/4': 22.,
           'M20': 22.,
           '7/8': 24.,
           'M22': 24.,
           'M24': 26.,
           '1': 27.,
           }

BOLT_Fu = {'A325M': 830.,
          'A490M': 1040.,
          'A325': 825.,
          'A490': 1035.,
          }

In [5]:
Params = Data()

Params.set( 'Bolts',
          type = 'A325',
          size = '3/4',
          hole_type = 'punched',  # 'punched' or 'drilled'
          Nrows = 2,              # a row is parallel to the load
          Nlines = 2,             # a line is perpendicular to the load
          pitch = 75.,            # spacing measure paprallel to load (along length of angle)
          end_distance = 30.,     # from end of angle to center of closest hole
          )

Bolts = Params.Bolts
Params.set('Bolts',
          Fu = BOLT_Fu[Bolts.type],
          d = BOLT_DIA[Bolts.size],
          ha = HOLE_DIA[Bolts.size] + (0. if Bolts.hole_type == 'drilled' else 2.),
          )

In [6]:
Dsg = 'L102x76x13'
A,d,b,t = SST.section(Dsg,'A,D,B,T')
Params.set('Angle',
           Fy = 350.,
           Fu = 450.,
           dsg = Dsg,
           Ag = A,
           d = d,
           b = b,
           t = t,
           bolted_leg = 'long',   # 'short' or 'long'
           m = 1,  
           threads_intercepted = True)
Params.namespaces

{'Angle': {'Ag': 2100.0,
  'Fu': 450.0,
  'Fy': 350.0,
  'b': 76.2,
  'bolted_leg': 'long',
  'd': 102.0,
  'dsg': 'L102x76x13',
  'm': 1,
  't': 12.7,
  'threads_intercepted': True},
 'Bolts': {'Fu': 825.0,
  'Nlines': 2,
  'Nrows': 2,
  'd': 19.049999999999997,
  'end_distance': 30.0,
  'ha': 24.0,
  'hole_type': 'punched',
  'pitch': 75.0,
  'size': '3/4',
  'type': 'A325'}}

In [7]:
raise Exception('Converted to Here!')

Exception: Converted to Here!

In [ ]:
class BoltedLegAngle(Designer):
        
    Fy = Param((300.,450.),value=345.)
    Fu = Param((450.,500.),value=450.)
    Fub = Param(BOLT_TYPES,description='bolt type')
    AngleDsg = Param('L102x76x13')
    bolt_size = Param(BOLT_SIZES,value='M20')
    hole_type = Param(['punched','drilled'],value='punched')
    pitch = Param((0.,75,1.),value=54.)           # if 0, will use minimum
    end_distance = Param((0.,50,.5),value=30)    # if 0, will use minimum
    threads_intercepted = Param(True)
    shear_type = Param(['single','double'],value='single')
    bolted_leg = Param(['long','short'],value='long')
    Nlines = Param((1,2),value=1)
    Nrows = Param((2,10),value=2)
    
des = BoltedLegAngle(var='Tr',units='kN',trace=True)
des.run()

In [ ]:
REQ = des.require   # useful abbreviations
CHK = des.check       
REC = des.record

### Sanity check of input data:

In [ ]:
REQ(bolt_size in BOLT_SIZES,'bolt size is not one of available sizes',
    bolt_size=bolt_size, BOLT_SIZES=BOLT_SIZES)
REQ(hole_type in ['punched','drilled'])
REQ(threads_intercepted in [False,True])
REQ(shear_type in ['single','double'])
REQ(bolted_leg in ['short','long'])
REQ(Nlines in [1,2])
REQ(Nrows >= 2 and Nrows <= 10)

In [ ]:
# data derived from input data:  pitch, end_distance, gauges, etc.

Ag,d,b,t = des.SST.section(AngleDsg,properties='A,D,B,T')
if bolted_leg == 'short':  # bolts go thru leg of size 'd'
    d,b = b,d
    
bolt_diameter = float(bolt_size[1:])
if pitch in [0,None]:
    pitch = 2.7*bolt_diameter
    
min_end_distance,min_edge_distance = table_search(bolt_diameter,Table6)
if Nrows <= 2:
    min_end_distance = 1.5*bolt_diameter  # CSA S16-01 22.3.4
if end_distance in [0,None]:
    end_distance = min_end_distance

if Nlines == 1:
    g2 = 0
    dmax,g1 = table_search(d-2,UG1)
elif Nlines == 2:
    dmax,g1,g2 = table_search(d-2,UG2)
edge_distance = d-(g1+g2)

### Check Bolting Details
Failure to meet requirements is not a fatal error. Results are reported in the work record.

In [ ]:
# CSA S16-09  22.3.1 thru .4:
max_edge_distance = min(12.*t,150.)
REQ(edge_distance > bolt_diameter/2.,'Angle leg of {0} mm does not support {1} lines of bolts.'.format(d,Nlines))

CHK(pitch >= 2.7*bolt_diameter,'Pitch','pitch',min_pitch=2.7*bolt_diameter)
CHK(edge_distance >= min_edge_distance,'Edge distance','edge_distance,min_edge_distance')
CHK(edge_distance <= max_edge_distance,'Edge distance','edge_distance,max_edge_distance')
CHK(end_distance >= min_end_distance,'End distance','end_distance,min_end_distance')

# CSA S16-09  28.4.1:
hole_allowance = hole_diameter = bolt_diameter + 2.
if hole_type == 'punched':
    CHK(t < bolt_diameter+4,'Punched holes','bolt_diameter,t',max_t=bolt_diameter+4)

# CSA S16-09  12.3.2:
if hole_type == 'punched':
    hole_allowance += 2.

### Strength Calculations

In [ ]:
# CSA S16-09  13.1
phi = 0.90
phiu = 0.75
phib = 0.80 
phibr = 0.80

#### Gross section yield:

In [ ]:
# CSA S16-09   13.2 (a) (i):
REC('Gross area yield','Ag',Tr=phi*Ag*Fy/1000.)

#### Net section fracture:

In [ ]:
# CSA S16-09   13.2 (a) (iii):
An = Ag - Nlines*hole_allowance*t
# CSA S16-09   12.3.3.2 (b)
if Nrows >= 4:
    Ane = 0.80*An
else:
    Ane = 0.60*An
REC('Net area fracture','An,Ane',Tr=phiu*Ane*Fu/1000.)

#### Block shear failure:
![Block Shear Patterns](bolted-single-angle-shear-blocks.svg)

In [ ]:
#  CSA S16-09    13.11
e = end_distance
L = (Nrows-1.)*pitch

# Case 1 - one shear area, tension failure from furthest bolt to edge
An = (d - g1 - (Nlines-0.5)*hole_allowance)*t
Agv = (e+L)*t
Ut = 0.6
Fv = (Fy+Fu)/2.
if Fy > 485:     # CSA S16-09   13.11  (foot note)
    Fv = Fy
REC('Block shear (case 1)','An,Agv,Ut',Tr=phiu*(Ut*An*Fu + 0.6*Agv*Fv)/1000.)

In [ ]:
# Case 2 - for 2 or more lines of bolts
if Nlines > 1:
    An = (g2 - hole_allowance)*t
    Agv = 2.*(e+L)*t
    Ut = 0.6
    REC('Block shear (case 2)','An,Agv,Ut',Tr=phiu*(Ut*An*Fu + 0.6*Agv*Fv)/1000.)

In [ ]:
# Case 3 - tearout
An = 0.
Agv = 2.*(e+L)*t*Nlines
Ut = 0.
REC('Block shear (tearout)','An,Agv,Ut',Tr=phiu*(Ut*An*Fu + 0.6*Agv*Fv)/1000.)

#### Fastener strength, bearing-type connection: bolts in shear

In [ ]:
m = 1
if shear_type == 'double':
    m = 2
n = Nrows*Nlines
db = bolt_diameter
L = (Nrows-1.)*pitch   # length of connection

# CSA S16-09   13.12.1.2 (c)
multiplier = 1.0
if L >= 15.*d:
    multiplier = max(0.75,1.075 - 0.005*L/d)
if threads_intercepted:
    multiplier *= 0.70
Vr = multiplier*0.6*phib*n*m*(3.1415926*db*db/4.)*Fub/1000.
REC('Bolt Shear',multiplier=multiplier,n=n,Tr=Vr)

#### Bolts in bearing:

In [ ]:
# CSA S16-09    13.12.1.2:
Br = 3.*phibr*n*t*db*Fu/1000.
REC('Bolt Bearing',n=n,d=db,t=t,Tr=Br)

#### Combined tearout and bearing:

In [ ]:
# combine tearout of bolts closest to end with bearing of remainder
e = end_distance
Agv = 2.*e*t*Nlines
n = Nlines*(Nrows-1)
Tr = phiu*(0.6*Agv*Fv)/1000.
Br = 3.*phibr*n*t*db*Fu/1000.
REC('Bolt bearing + end tearout',n=n,t=t,d=db,Agv=Agv,Tr=Tr+Br)

### Summary:

In [ ]:
des.show('Fy,Fu,AngleDsg,Ag,d,b,t,Fub,bolt_size,bolt_diameter,hole_type',
         'pitch,end_distance,threads_intercepted,shear_type,bolted_leg,Nlines,Nrows',
         'end_distance,edge_distance,g1,g2')
des.summary()          